In [1]:
import pandas as pd
# Import SQLAlchemy `automap` and other dependencies here
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [4]:
# Create an engine for the `emoji.sqlite` database
engine = create_engine("sqlite:///ncaa_Rank_Seed2018.sqlite", echo=False)

In [5]:
# Use the Inspector to explore the database and print the table names
inspector = inspect(engine)
inspector.get_table_names()

['ranks', 'seeds']

In [6]:
# Use Inspector to print the column names and types
columns = inspector.get_columns('ranks')
for c in columns:
    print(c['name'], c["type"])

Rank INTEGER
Name VARCHAR(255)
Team VARCHAR(255)
PTS INTEGER


In [16]:
# Use Inspector to print the column names and types
columns = inspector.get_columns('seeds')
for c in columns:
    print(c['name'], c["type"])

index INTEGER
Seed INTEGER
Name VARCHAR(255)
Region VARCHAR(255)


In [7]:
engine.execute('SELECT * FROM ranks LIMIT 10').fetchall()

[(1, 'Mike Daum', 'South Dakota St.', 314),
 (2, 'Justin Wright-Foreman', 'Hofstra', 283),
 (3, 'Chris Clemons', 'Campbell', 279),
 (4, 'Antoine Davis', 'Detroit Mercy', 274),
 (5, 'Yoeli Childs', 'BYU', 273),
 (6, 'Jermaine Marrow', 'Hampton', 259),
 (7, 'Bubba Parham', 'VMI', 258),
 (8, 'Carsen Edwards', 'Purdue', 255),
 (9, 'Jordan Ford', "Saint Mary's (CA)", 248),
 (10, 'David Jenkins', 'South Dakota St.', 248)]

In [12]:
engine.execute('SELECT * FROM seeds LIMIT 10').fetchall()

[(0, 1, 'Virginia', 'SOUTH'),
 (1, 16, 'UMBC', 'SOUTH'),
 (2, 8, 'Creighton', 'SOUTH'),
 (3, 9, 'Kansas St.', 'SOUTH'),
 (4, 5, 'Kentucky', 'SOUTH'),
 (5, 12, 'Davidson', 'SOUTH'),
 (6, 4, 'Arizona', 'SOUTH'),
 (7, 13, 'Buffalo', 'SOUTH'),
 (8, 6, 'Miami (FL)', 'SOUTH'),
 (9, 11, 'Loyola (IL)', 'SOUTH')]

In [13]:
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=True)
Ranks = Base.classes.ranks
Seeds = Base.classes.seeds

In [14]:
# Start a session to query the database
session = Session(engine)

In [15]:
# query the database
session.query(Ranks.Rank, Ranks.Name, Ranks.Team,Ranks.PTS).first()

(1, 'Mike Daum', 'South Dakota St.', 314)

In [24]:
#Display the teams from region
def Region_teams(region_name):
    results = session.query(Seeds.Seed, Seeds.Name).filter(Seeds.Region == region_name).\
    order_by(Seeds.Seed.asc()).all()
    return results


In [25]:
south_results = Region_teams("SOUTH")
print(south_results)

[(1, 'Virginia'), (2, 'Cincinnati'), (3, 'Tennessee'), (4, 'Arizona'), (5, 'Kentucky'), (6, 'Miami (FL)'), (7, 'Nevada'), (8, 'Creighton'), (9, 'Kansas St.'), (10, 'Texas'), (11, 'Loyola (IL)'), (12, 'Davidson'), (13, 'Buffalo'), (14, 'Wright St.'), (15, 'Georgia St.'), (16, 'UMBC')]


In [26]:
east_results = Region_teams("EAST")
print(east_results)

[(1, 'Villanova'), (2, 'Purdue'), (3, 'Texas Tech'), (4, 'Wichita St.'), (5, 'West Virginia'), (6, 'Florida'), (7, 'Arkansas'), (8, 'Virginia Tech'), (9, 'Alabama'), (10, 'Butler'), (11, 'St. Bonaventure'), (11, 'UCLA'), (12, 'Murray St.'), (13, 'Marshall'), (14, 'Stephen F. Austin'), (15, 'Cal St. Fullerton'), (16, 'Radford'), (16, 'Long Island')]


In [27]:
west_results = Region_teams("WEST")
print(west_results)

[(1, 'Xavier'), (2, 'North Carolina'), (3, 'Michigan'), (4, 'Gonzaga'), (5, 'Ohio St.'), (6, 'Houston'), (7, 'Texas A&M'), (8, 'Missouri'), (9, 'Florida St.'), (10, 'Providence'), (11, 'San Diego St.'), (12, 'South Dakota St.'), (13, 'UNC Greensboro'), (14, 'Montana'), (15, 'Lipscomb'), (16, 'Texas Southern'), (16, 'N.C. Central')]


In [28]:
midwest_results = Region_teams("MIDWEST")
print(midwest_results)

[(1, 'Kansas'), (2, 'Duke'), (3, 'Michigan St.'), (4, 'Auburn'), (5, 'Clemson'), (6, 'TCU'), (7, 'Rhode Island'), (8, 'Seton Hall'), (9, 'North Carolina St.'), (10, 'Oklahoma'), (11, 'Syracuse'), (11, 'Arizona St.'), (12, 'New Mexico St.'), (13, 'Col. of Charleston'), (14, 'Bucknell'), (15, 'Iona'), (16, 'Penn')]


In [35]:
#Rank, Name, Team, PTS
top_25 = session.query(Ranks.Rank, Ranks.Name, Ranks.PTS).limit(300).all()
print(len(top_25))

254
